In [2]:
#NOTE: an explanation of the relevant method here is in the README file for this repository

#FINAL METHOD
#change method based on duration_hours and fractional_duration
#change time-bin size and size of the phase mask
#catch AttributeError and TimeDivisionError
#uses minmax normalization for all curves
#downloads curves to a file on desktop; closes figures once completer 

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)

path = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves/"
author = ""

for x in range(4100, 4200):
    try:
        TOI_name = "TIC " + rows[x][fields.index("TIC ID")] 
        period = float(rows[x][fields.index("Period (days)")])
        t0 = float(rows[x][fields.index("Epoch (BJD)")])
        duration_hours = float(rows[x][fields.index("Duration (hours)")])
        fractional_duration = (duration_hours / 24.0) / period

        if(duration_hours < 3.0): 
            
            if(fractional_duration < 0.004): 
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
            
            if(fractional_duration < 0.01):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -15*fractional_duration) & (temp_fold.phase < 15.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0044) 
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
            
            elif(fractional_duration < 0.04):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -4*fractional_duration) & (temp_fold.phase < 4.0*fractional_duration) 
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
            
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008)

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))

                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
    
        elif(duration_hours < 5.0):
            search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
            author = "SPOC"
            if(len(search_result) == 0):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                    author = "QLP"

            lcs = search_result.download_all() 
            lc_raw = lcs.stitch() 

            lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

            temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

            fractional_duration = (duration_hours / 24.0) / period

            phase_mask = (temp_fold.phase > -25*fractional_duration) & (temp_fold.phase < 25.0*fractional_duration) 
            lc_zoom = temp_fold[phase_mask]

            lc_local = lc_zoom.bin(time_bin_size=0.0008)
            lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            fig, axes = plt.subplots()
            lc_local.scatter(ax=axes, color="black")
            fig.savefig(path + "fig" + str(x) + author + ".pdf")
            plt.close()
        
        else:
            if(fractional_duration < 0.02):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
                
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008) 

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.scatter(ax=axes, color="black")
                fig.savefig(path + "fig" + str(x) + author + ".pdf")
                plt.close()
    
    except AttributeError:
        print("AttributeError occured with", TOI_name, "item", str(x))
    except ZeroDivisionError:
        print("ZeroDivisionError occured with", TOI_name, "item", str(x))
    except:
        print("An error occured with", TOI_name, "item", str(x))

No data found for target "TIC 235684392".
No data found for target "TIC 235684392".
No data found for target "TIC 233719215".
No data found for target "TIC 233719215".
No data found for target "TIC 233053554".
No data found for target "TIC 233053554".
/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/collections.py:212: LightkurveWarning: The following columns will be excluded from stitching because the column types are incompatible: {'kspsap_flux_lag'}
  warnings.warn(
No data found for target "TIC 232567319".
No data found for target "TIC 230388132".
No data found for target "TIC 230388132".
No data found for target "TIC 229639351".
No data found for target "TIC 229639351".
No data found for target "TIC 229605891".
No data found for target "TIC 229605891".
No data found for target "TIC 229508727".
No data found for target "TIC 229508727".
No data found for target "TIC 224601047".
No data found for target "TIC 224601047".
No data found for target "TIC 21986

An error occured with TIC 230006888 item 4158
An error occured with TIC 229786610 item 4160


No data found for target "TIC 229408913".
No data found for target "TIC 229408913".
/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/collections.py:212: LightkurveWarning: The following columns will be excluded from stitching because the column types are incompatible: {'sap_flux', 'kspsap_flux', 'kspsap_flux_sml', 'kspsap_flux_lag'}
  warnings.warn(
No data found for target "TIC 199610140".
No data found for target "TIC 198616607".
No data found for target "TIC 198616607".
No data found for target "TIC 193773468".
No data found for target "TIC 193773468".
No data found for target "TIC 160618074".
No data found for target "TIC 160618074".
No data found for target "TIC 159438097".
No data found for target "TIC 157218656".
No data found for target "TIC 156215204".
No data found for target "TIC 156215204".
No data found for target "TIC 153896150".
No data found for target "TIC 150761163".
No data found for target "TIC 150761163".
No data found for target "TIC 1

An error occured with TIC 141488193 item 4174


No data found for target "TIC 123357034".
No data found for target "TIC 71396541".
No data found for target "TIC 47937361".
No data found for target "TIC 47937361".
No data found for target "TIC 468396217".
No data found for target "TIC 468396217".
No data found for target "TIC 288471040".
No data found for target "TIC 420680343".
No data found for target "TIC 417646390".
No data found for target "TIC 408729765".
No data found for target "TIC 408729765".
No data found for target "TIC 400093775".
No data found for target "TIC 400093775".
No data found for target "TIC 39573090".
No data found for target "TIC 39573090".
No data found for target "TIC 371711170".
No data found for target "TIC 371711170".
No data found for target "TIC 279947414".
No data found for target "TIC 138329479".
No data found for target "TIC 138329479".
No data found for target "TIC 137157546".
No data found for target "TIC 103096524".
No data found for target "TIC 103096524".
No data found for target "TIC 102719996

In [1]:
#NOTE: an explanation of the relevant method here is in the README file for this repository

#FINAL METHOD
#change method based on duration_hours and fractional_duration
#change size of the phase mask
#catch AttributeError and TimeDivisionError
#uses minmax normalization for all curves
#downloads .csv file of all points to a folder on computer; closes figures once completed 

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)

path = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves/"
author = ""

for x in range(4466, 4800):
    try:
        TOI_name = "TIC " + rows[x][fields.index("TIC ID")] 
        period = float(rows[x][fields.index("Period (days)")])
        t0 = float(rows[x][fields.index("Epoch (BJD)")])
        duration_hours = float(rows[x][fields.index("Duration (hours)")])
        fractional_duration = (duration_hours / 24.0) / period

        if(duration_hours < 3.0): 
            
            if(fractional_duration < 0.004): 
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
            
            if(fractional_duration < 0.01):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -15*fractional_duration) & (temp_fold.phase < 15.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008) 
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
            
            elif(fractional_duration < 0.04):
                search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                author = "SPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                    author = "TESSSPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -4*fractional_duration) & (temp_fold.phase < 4.0*fractional_duration) 
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
            
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008)

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))

                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
    
        elif(duration_hours < 5.0):
            search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
            author = "SPOC"
            if(len(search_result) == 0):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                    author = "QLP"

            lcs = search_result.download_all() 
            lc_raw = lcs.stitch() 

            lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

            temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

            fractional_duration = (duration_hours / 24.0) / period

            phase_mask = (temp_fold.phase > -25*fractional_duration) & (temp_fold.phase < 25.0*fractional_duration) 
            lc_zoom = temp_fold[phase_mask]

            lc_local = lc_zoom.bin(time_bin_size=0.0008)
            lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            fig, axes = plt.subplots()
            lc_local.plot(ax=axes, color="black")
            line = fig.gca().lines[0]
            data = np.asarray([line.get_xdata(), line.get_ydata()])
            filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
            np.savetxt(filename, data, delimiter=',')
            plt.close()
        
        else:
            if(fractional_duration < 0.02):
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
                
            else:
                search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
                author = "TESSSPOC"
                if(len(search_result) == 0):
                    search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
                    author = "SPOC"
                    if(len(search_result) == 0):
                        search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                        author = "QLP"

                lcs = search_result.download_all() 
                lc_raw = lcs.stitch() 

                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4) 

                temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

                fractional_duration = (duration_hours / 24.0) / period

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                lc_global = lc_fold.bin(time_bin_size=0.0008) 

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=0.0008)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
                fig, axes = plt.subplots()
                lc_local.plot(ax=axes, color="black")
                line = fig.gca().lines[0]
                data = np.asarray([line.get_xdata(), line.get_ydata()])
                filename = path + "fig" + str(x) + "_" + TOI_name[4:] + "_" + author + ".csv"
                np.savetxt(filename, data, delimiter=',')
                plt.close()
    
    except AttributeError:
        print("AttributeError occured with", TOI_name, "item", str(x))
    except ZeroDivisionError:
        print("ZeroDivisionError occured with", TOI_name, "item", str(x))
    except:
        print("An error occured with", TOI_name, "item", str(x))

No data found for target "TIC 1405997635".
No data found for target "TIC 1405997635".
No data found for target "TIC 1405997635".
/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/search.py:414: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


AttributeError occured with TIC 1405997635 item 4466


No data found for target "TIC 1234810324".
No data found for target "TIC 1234810324".
No data found for target "TIC 1234810324".
/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/search.py:414: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


AttributeError occured with TIC 1234810324 item 4467


No data found for target "TIC 208297760".
No data found for target "TIC 263748033".
No data found for target "TIC 326386495".
No data found for target "TIC 106940971".
No data found for target "TIC 271042217".
No data found for target "TIC 362222177".


...

No data found for target "TIC 438149332".
No data found for target "TIC 438149332".
No data found for target "TIC 280292434".
No data found for target "TIC 280292434".
No data found for target "TIC 265619984".
No data found for target "TIC 265619984".
No data found for target "TIC 142532090".
No data found for target "TIC 72097808".
No data found for target "TIC 72097808".
/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/search.py:414: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


AttributeError occured with TIC 72097808 item 4798


No data found for target "TIC 291198795".
No data found for target "TIC 291198795".


AttributeError occured with TIC 291198795 item 4799


/Users/anahitasrinivasan/opt/anaconda3/lib/python3.9/site-packages/lightkurve/search.py:414: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(
